# Ionospheric Connection Explorer (ICON) on Jupyter Notebook v4.0

<img src="https://drive.google.com/uc?id=1smTGLEQl2fu84PE8PQroytadcd5cT0SG"> 


- Access all data products and data documents on the website: <https://icon.ssl.berkeley.edu/Data>
- <p><a href="https://icon.ssl.berkeley.edu/Portals/devIcon/Data/Rules_of_The_Road_ICON_v0.2.pdf?ver=2018-05-07-141215-957" target="_blank">ICON Rules of the Road</a> - Please review the RotR for the ICON products, and consult with instrument&nbsp;scientists to support your analyses.</p>
- <p><a href="ftp://icon-science.ssl.berkeley.edu/pub" target="_blank">ICON Public Data</a> - All ICON Data Products are available in NetCDF formats at this FTP site.</p>


LEVEL.2 Data products: 
- L2.1 MIGHTI -- Line-of-Sight Winds 
- L2.2 MIGHTI -- Neutral Vector Winds -- Zonal Wind, Meridional Wind, Field-aligned Wind 
- L2.3 MIGHTI-A/B* -- Neutral Temperature
- L2.4 FUV--Column O/$N_{2}$
- L2.5 FUV--Nighttime $O^{+}$ Density, NmF2, HmF2
- L2.6 EUV--Daytime $O^{+}$ Density, NmF2, HmF2
- L2.7 IVM-A/B* -- Ion Drift, Ion Densities, Ion Temperature 
- A/B* are two identical instruments pointing to different directions.

### Note: 
- IDL version:
> <p><a href=" https://github.com/YJWu-SSL/ICON_Data_Demo/blob/master/ICON_IDL_EUVIVM_Tutorial.pro" target="_blank">ICON_IDL_EUVIVM_Tutorial.pro</a>
> <p><a href=" https://github.com/YJWu-SSL/ICON_Data_Demo/blob/master/ICON_IDL_MIGHTIIVM_Tutorial.pro" target="_blank">ICON_IDL_MIGHTIIVM_Tutorial.pro</a>
> <p><a href=" https://github.com/YJWu-SSL/ICON_Data_Demo/blob/master/ICON_IDL_MIGHTI_Tutorial.pro" target="_blank">ICON_IDL_MIGHTI_Tutorial.pro</a>
- ICON demo on InGeo: https://ingeo.datatransport.org/v3/workspace Go To examples > tutorials
- Please read variable notes and quality flags for caveats, limitations, and best practices.
- Please check official data sources for the latest version of the data.
- Page Maintanence: Yen-Jung Wu yjwu@dn {dn=ssl.berkeley.edu}
- Code Contributor: Dr. Colin Triplett triplett@dn and Dr. Brian Harding bharding@dn


# Why Colab? Where is Colab running? How to run the script locally?


1. According to https://colab.research.google.com/notebooks/intro.ipynb
, Colaboratory, or "Colab" for short, allows you to write and execute Python in your browser, with 
> - Zero configuration required
> - Free access to GPUs
> - Easy sharing


2. Google Colab provides free but limited GPUs, runtimes, and memory. Each runtime will be terminated after 12 hours.

3. Because of the runtime limit, running the script on Colab (free version) directly is only good for demonstration and quick view purposes, but not effective for any scientific analysis.
> - Download the notebook and run on your local machine.
> - Same notebook for running locally: 
https://github.com/YJWu-SSL/ICON_Data_Demo/blob/master/ICON_data_demo_Local_v02.ipynb


#1. Get Ready

### Import Python packages

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, MinuteLocator
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator
from datetime import datetime, timedelta
import bisect
import pandas as pd
import seaborn as sns
import pylab
import os
import glob
from ftplib import FTP
import fnmatch
import matplotlib.dates as mdates
import xarray as xr

'''Optional: Specify the plotting style '''
plt.style.use('seaborn')
plt.rcParams.update({'font.size': 15,\
                     'xtick.labelsize' : 15,\
                     'ytick.labelsize' : 15,\
                     'axes.titlesize' : 16,\
                     'axes.labelsize' : 16,\
                     'date.autoformatter.minute': '%H:%M' })

### Additional functions

In [2]:
def download_ICON_ftp(ftp, target_YYYY, target_doy, ICON_product, ICON_instrument, ICON_level='LEVEL.2',MTAB=None):
  '''
  INPUTS
  ------

      ftp        class : ftp class that has logged in to the ftp site ftp=libftp.ftp('address')
      target_YYYY        str : the year of the target day 
      target_doy         str : the day of year of the target day
      ICON_product       str : i.e. 'L2.1','L2.2','L2.3','L2.4','L2.5','L2.6','L2.7'
      ICON_instrument    str : has to be one of the following ['MIGHTI','IVM-A','FUV','EUV']

      Optional:
        ICON_level       str : 'LEVEL.2','LEVEL.1'
        MTAB             str : A required input when retrieving L2.3 MIGHTI Temperature, MTAB='A' for MIGHTI-A, MTAB='B' for MIGHTI-B 

  RESULT
  -------
      Download (retrieve) the latest version of ICON data from the FTP site hosting at UC Berkeley (https://icon.ssl.berkeley.edu/Data)

  '''
 
  target_dir='/pub/%s/%s/%s/%s' % (ICON_level, ICON_instrument,target_YYYY,target_doy)

  if target_dir in ftp.nlst('/pub/%s/%s/%s/' % (ICON_level, ICON_instrument,target_YYYY)):
    ftp.cwd(target_dir+'/ZIP/')
    filenames=ftp.nlst()
    if not (ICON_product == 'L2-3'):
      if ftp.nlst("ICON_%s*" % (ICON_product)):
        select_fn=sorted(ftp.nlst("ICON_%s*" % (ICON_product)), key=lambda x: ftp.voidcmd(f"MDTM {x}"))[-1]
      else:
        raise Exception('File does not exist!')
    else:
      if (MTAB == None):
        raise Exception('Please specify MIGHTI-A or -B in keyword MTAB, MTAB="A" for example ')
      else:
        if ftp.nlst('ICON_%s_%s-%s*' % (ICON_product, ICON_instrument,MTAB)):
          select_fn=sorted(ftp.nlst('ICON_%s_%s-%s*' % (ICON_product, ICON_instrument,MTAB)), key=lambda x: ftp.voidcmd(f"MDTM {x}"))[-1]
        else:
          raise Exception('File does not exist!')
    download_file = open(select_fn, 'wb')
    ftp.retrbinary('RETR '+ select_fn, download_file.write)

    download_file.close()
    print('%s has been retrieved' % select_fn)
  else:
    raise Exception('/pub/%s/%s/%s/%s/ZIP/ does not exist!' % (ICON_level, ICON_instrument,target_YYYY,target_doy))
  
  return

class OOMFormatter(matplotlib.ticker.ScalarFormatter):
  '''
  Math text formatter on colorbars
  source:
  https://stackoverflow.com/questions/42656139/set-scientific-notation-with-fixed-exponent-and-significant-digits-for-multiple
  '''
  def __init__(self, order=0, fformat="%1.1f", offset=True, mathText=True):
      self.oom = order
      self.fformat = fformat
      matplotlib.ticker.ScalarFormatter.__init__(self,useOffset=offset,useMathText=mathText)
  def _set_order_of_magnitude(self):
      self.orderOfMagnitude = self.oom
  def _set_format(self, vmin=None, vmax=None):
      self.format = self.fformat
      if self._useMathText:
            self.format = r'$\mathdefault{%s}$' % self.format

# 2. Download ICON data from the FTP site

In [15]:
#'''Insert a target day'''
target_datetime=datetime(2020,4,20) # (year, month, day)
target_YYYY='%4i' % target_datetime.year
target_mm='%02i' % target_datetime.month
target_dd='%02i' % target_datetime.day
target_doy='%03i' % ((target_datetime-datetime(np.int(target_YYYY),1,1)).days+1)
target_date_str='%s-%s-%s' % (target_YYYY,target_mm,target_dd)
#
ftp=FTP('icon-science.ssl.berkeley.edu')
#
ftp.login()
download_ICON_ftp(ftp, target_YYYY, target_doy, ICON_product='L2-2', ICON_instrument='MIGHTI')
download_ICON_ftp(ftp, target_YYYY, target_doy, ICON_product='L2-3', ICON_instrument='MIGHTI',MTAB='A')
download_ICON_ftp(ftp, target_YYYY, target_doy, ICON_product='L2-4', ICON_instrument='FUV')
download_ICON_ftp(ftp, target_YYYY, target_doy, ICON_product='L2-5', ICON_instrument='FUV')
download_ICON_ftp(ftp, target_YYYY, target_doy, ICON_product='L2-6', ICON_instrument='EUV')
download_ICON_ftp(ftp, target_YYYY, target_doy, ICON_product='L2-7', ICON_instrument='IVM-A')
## 
ftp.quit()

/tmp/ipykernel_24/2116705907.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  target_doy='%03i' % ((target_datetime-datetime(np.int(target_YYYY),1,1)).days+1)


ICON_L2-2_MIGHTI_2020-04-20_v04r001.ZIP has been retrieved
ICON_L2-3_MIGHTI-A_Temperature_2020-04-20_v05r000.ZIP has been retrieved
ICON_L2-4_FUV_Day_2020-04-20_v04r000.ZIP has been retrieved
ICON_L2-5_FUV_Night_2020-04-20_v05r001.ZIP has been retrieved
ICON_L2-6_EUV_2020-04-20_v03r000.ZIP has been retrieved
ICON_L2-7_IVM-A_2020-04-20_v06r000.ZIP has been retrieved


'221 Goodbye.'

In [3]:
#'''Insert a target day'''
target_datetime=datetime(2020,4,20) # (year, month, day)
target_YYYY='%4i' % target_datetime.year
target_mm='%02i' % target_datetime.month
target_dd='%02i' % target_datetime.day
target_doy='%03i' % ((target_datetime-datetime(np.int(target_YYYY),1,1)).days+1)
target_date_str='%s-%s-%s' % (target_YYYY,target_mm,target_dd)

/tmp/ipykernel_24/368613184.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  target_doy='%03i' % ((target_datetime-datetime(np.int(target_YYYY),1,1)).days+1)


In [ ]:
'''Unzip the file'''
!unzip '*.ZIP'

Archive:  ICON_L2-2_MIGHTI_2020-04-20_v04r001.ZIP
ICON archive of file set ICON_L2-2_MIGHTI_2020-04-20.
replace 111/ICON_L2-2_MIGHTI_Vector-Wind-Green_2020-04-20_v04r001.NC? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [14]:
#df = pd.read_csv('ICON_L2-7_IVM-A_2020-04-20_v06r000.ZIP')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

#3. Read ICON netCDF files

In [ ]:
'''MIGHTI L2.2 Vector Wind'''
MT_color= 'Green'
fn_L22=glob.glob('/content/%s/ICON_L2-2_*%s*.NC' % (target_doy,MT_color))[0]

dm_g=xr.open_dataset(fn_L22)
dm_g=dm_g.rename({'Epoch': 'time_ms',
                 'ICON_L22_Orbit_Number':'orb_num',
                 'ICON_L22_Longitude':'tang_lon',
                 'ICON_L22_Latitude':'tang_lat',
                 'ICON_L22_Altitude':'tang_alt',
                 'ICON_L22_Meridional_Wind': 'umer',
                 'ICON_L22_Zonal_Wind':'uzon',
                 'ICON_L22_Local_Solar_Time':'tang_slt',
                 'ICON_L22_Wind_Quality':'wind_quality'})

dm_g['time']=pd.to_datetime(np.array([datetime(1970,1,1) + timedelta(seconds = 1e-3*s) for s in dm_g['time_ms'].values]))
# Only allow the good data: dm_g.wind_quality >= 0.5
good_data=(dm_g.wind_quality == 0.5)|(dm_g.wind_quality == 1)
dm_g[['umer','uzon']]=dm_g[['umer','uzon']].where(good_data)


print('Orbit numbers: %04i to %04i' % (np.nanmin(dm_g.orb_num),np.nanmax(dm_g.orb_num)))

In [9]:
# Check out more information in the NC file
dm_g

NameError: name 'dm_g' is not defined

In [ ]:
'''MIGHTI L2.2 Vector Wind'''
MT_color= 'Red'
fn_L22=glob.glob('/content/%s/ICON_L2-2_*%s*.NC' % (target_doy,MT_color))[0]
### Load winds

dm_r=xr.open_dataset(fn_L22)
dm_r=dm_r.rename({'Epoch': 'time_ms',
                 'ICON_L22_Orbit_Number':'orb_num',
                 'ICON_L22_Longitude':'tang_lon',
                 'ICON_L22_Latitude':'tang_lat',
                 'ICON_L22_Altitude':'tang_alt',
                 'ICON_L22_Meridional_Wind': 'umer',
                 'ICON_L22_Zonal_Wind':'uzon',
                 'ICON_L22_Local_Solar_Time':'tang_slt',
                 'ICON_L22_Wind_Quality':'wind_quality'})

good_data = (dm_r.wind_quality == 1)| (dm_r.wind_quality == 0.5)
dm_r['time']=pd.to_datetime(np.array([datetime(1970,1,1) + timedelta(seconds = 1e-3*s) for s in dm_r['time_ms'].values]))
dm_r[['umer','uzon']]=dm_r[['umer','uzon']].where(good_data)

print('Orbit numbers: %04i to %04i' % (np.nanmin(dm_r.orb_num),np.nanmax(dm_r.orb_num)))


In [ ]:
'''MIGHTI L2.3 Temperature'''
MTAB='A'

fn_L23=glob.glob('/content/%s/ICON_L2-3_MIGHTI-%s*.NC' % (target_doy, MTAB))[0]


dTN=xr.open_dataset(fn_L23)
dTN=dTN.rename({'Epoch': 'time_ms',
                 'ICON_L23_Orbit_Number':'orb_num',
                 'ICON_L23_MIGHTI_%s_Tangent_Longitude' % (MTAB):'tang_lon',
                 'ICON_L23_MIGHTI_%s_Tangent_Latitude' % (MTAB):'tang_lat',
                 'ICON_L23_MIGHTI_%s_Tangent_Altitude' % (MTAB):'tang_alt',
                 'ICON_L23_MIGHTI_%s_Temperature' % (MTAB): 'TN',
                 'ICON_L23_MIGHTI_%s_Tangent_Local_Solar_Time' % (MTAB):'tang_slt',
                 'ICON_L1_MIGHTI_%s_Quality_Flag_South_Atlantic_Anomaly' % (MTAB):'saa_flag',
                 'ICON_L1_MIGHTI_%s_Quality_Flag_Bad_Calibration' % (MTAB):'cali_flag'
                 })

good_data = (dTN.saa_flag == 0) & (dTN.cali_flag == 0) & (dTN.TN > 0) & (abs(dTN.TN) < 10000)
dTN['time']=pd.to_datetime(np.array([datetime(1970,1,1) + timedelta(seconds = 1e-3*s) for s in dTN['time_ms'].values]))
dTN['TN']=dTN['TN'].where(good_data)
print('Orbit numbers: %04i to %04i' % (np.nanmin(dTN.orb_num),np.nanmax(dTN.orb_num)))


In [ ]:
'''FUV L2.4 Disk O/N2'''
fn_L24 =glob.glob('/content/%s/ICON_L2-4_FUV*.NC' % (target_doy))[0]

#  Load L2.4 disk O/N2 #

dON2=xr.open_dataset(fn_L24)
dON2=dON2.rename({'Epoch': 'time_ms',
                 'ICON_L24_Disk_ON2': 'disk_ON2',
                 'ICON_L24_Local_Solar_Time_Disk': 'disk_slt',
                 'ICON_L24_Disk_Longitude': 'disk_lon',
                 'ICON_L24_Disk_Latitude': 'disk_lat',
                 'ICON_L24_Disk_Magnetic_Longitude': 'disk_mlon',
                 'ICON_L24_Disk_Magnetic_Latitude': 'disk_mlat',
                 'ICON_L24_Disk_Retrieval_Flags': 'model_flags'
                 })

good_data = (dON2.model_flags == 0)
dON2['time']=pd.to_datetime(np.array([datetime(1970,1,1) + timedelta(seconds = 1e-3*s) for s in dON2['time_ms'].values]))
dON2['disk_ON2']=dON2['disk_ON2'].where(good_data)

print( (np.nanmin(dON2.time),np.nanmax(dON2.time)))



In [ ]:
'''FUV L2.5 Nighttime Ionosphere'''
fn_L25 =glob.glob('/content/%s/ICON_L2-5_FUV*.NC' % (target_doy))[0]

# Load L2.5 Nighttime plasma observations #
dL25=xr.open_dataset(fn_L25)
dL25=dL25.rename({'Epoch': 'time_ms',
                  'ICON_L25_HMF2': 'HmF2',
                  'ICON_L25_NMF2': 'NmF2',
                  'ICON_L25_O_Plus_Density': 'O_plus',
                  'ICON_L25_Orbit_Number': 'orb_num',
                  'ICON_L25_Local_Solar_Time': 'tang_slt',
                  'ICON_L25_Longitude': 'HmF2_lon',
                  'ICON_L25_Latitude': 'HmF2_lat',
                  'ICON_L25_Magnetic_Longitude': 'HmF2_mlon',
                  'ICON_L25_Magnetic_Latitude': 'HmF2_mlat',
                  'ICON_L25_Quality': 'L25_flag'
                  })

dL25['time']  = pd.to_datetime(np.array([datetime(1970,1,1) + timedelta(seconds = 1e-3*s) for s in dL25['time_ms'].values]))
good_data = (dL25.L25_flag== 1)# 
dL25[['HmF2','NmF2']]=dL25[['HmF2','NmF2']].where(good_data)
print('Orbit number: %04i to %04i' % (np.nanmin(dL25.orb_num),np.nanmax(dL25.orb_num)))

In [ ]:
'''EUV L2.6 Daytime Ionosphere'''
fn_L26 =glob.glob('/content/%s/ICON_L2-6_EUV*.NC' % (target_doy))[0]

# Load L2.6 Daytime plasma observations #
dEUV =xr.open_dataset(fn_L26)

dEUV=dEUV.rename({'Epoch': 'time_ms',
                  'ICON_L26_HmF2': 'HmF2',
                  'ICON_L26_NmF2': 'NmF2',
                  'ICON_L26_Oplus': 'O_plus',
                  'ICON_L26_Orbit_Number': 'orb_num',
                  'ICON_L26_Local_Solar_Time': 'tang_slt',
                  'ICON_L26_Longitude': 'tang_lon',
                  'ICON_L26_Latitude': 'tang_lat',
                  'ICON_L26_Altitude': 'tang_alt',
                  'ICON_L26_Magnetic_Longitude': 'tang_mlon',
                  'ICON_L26_Magnetic_Latitude': 'tang_mlat',
                  'ICON_L26_Flag': 'L26_flag'
                  })

dEUV['time']  = pd.to_datetime(np.array([datetime(1970,1,1) + timedelta(seconds = 1e-3*s) for s in dEUV['time_ms'].values]))
good_data = (dEUV.L26_flag == 1) | (dEUV.L26_flag == 0)# 
# dEUV=dEUV.where(good_data)
dEUV[['NmF2','HmF2','O_plus']]=dEUV[['NmF2','HmF2','O_plus']].where(good_data)

print('Orbit number: %04i to %04i' % (np.nanmin(dEUV.orb_num),np.nanmax(dEUV.orb_num)))


In [ ]:
'''IVM L2.7 In-situ Plasma Measurement'''
fn_L27 =glob.glob('/content/%s/ICON_L2-7_IVM-A*.NC' % (target_doy))[0]
# #### Load ion velocity
di =xr.open_dataset(fn_L27)

di=di.rename({'Epoch': 'time_ms',
                  'ICON_L27_Ion_Velocity_Meridional': 'vmer',
                  'ICON_L27_Ion_Velocity_Zonal': 'vzon',
                  'ICON_L27_Ion_Density': 'Ni',
                  'ICON_L27_Orbit_Number': 'orb_num',
                  'ICON_L27_Solar_Local_Time': 'slt',
                  'ICON_L27_Longitude': 'lon',
                  'ICON_L27_Latitude': 'lat',
                  'ICON_L27_Altitude': 'alt',
                  'ICON_L27_Magnetic_Longitude': 'mlon',
                  'ICON_L27_Magnetic_Latitude': 'mlat',
                  'ICON_L27_DM_Flag': 'DM_flag',
                  'ICON_L27_RPA_Flag': 'RPA_flag'
                  })

di['time']  = pd.to_datetime(np.array([datetime(1970,1,1) + timedelta(seconds = 1e-3*s) for s in di['time_ms'].values]))
good_data = (di.DM_flag == 0) & (di.RPA_flag < 2)# 
di[['vmer','vzon','Ni']]=di[['vmer','vzon','Ni']].where(good_data)

print('Orbit number: %04i to %04i' % (np.nanmin(di['orb_num']),np.nanmax(di['orb_num'])))


# Example 1: Visualizing data in a single orbit


*   This practice will demostrate the how to use *orbit number* and how to compare different ICON product.


*   Use L2.2 Vector Wind and L2.3 Temperature for demonstration.





In [ ]:
# Select a particular orbit
target_orbit_num = 2870

dTN_select_orb_num_idx = np.argwhere(dTN['orb_num'].values== target_orbit_num)
dTN_select_idx = dTN_select_orb_num_idx[:,0]

select_dTN = {}
select_dTN['orb_num'] = dTN['orb_num'][dTN_select_idx]
select_dTN['time']    = dTN['time'][dTN_select_idx]
select_dTN['time_ms']    = dTN['time_ms'][dTN_select_idx]
select_dTN['TN']      = dTN['TN'][dTN_select_idx]
select_dTN['tang_slt']      = dTN['tang_slt'][dTN_select_idx]
select_dTN['tang_lon'] = dTN['tang_lon'][dTN_select_idx]
select_dTN['tang_lat'] = dTN['tang_lat'][dTN_select_idx]
select_dTN['tang_alt'] = dTN['tang_alt'][dTN_select_idx]


dm_select_idx=np.where((dm_g['time']>=select_dTN['time'][0]) & (dm_g['time'] < select_dTN['time'][-1]))
select_dm_g = {}
select_dm_g['orb_num'] = dm_g['orb_num'][dm_select_idx]
select_dm_g['time']    = dm_g['time'][dm_select_idx]
select_dm_g['uzon']    = dm_g['uzon'][dm_select_idx]
select_dm_g['umer']    = dm_g['umer'][dm_select_idx]
select_dm_g['tang_slt']      = dm_g['tang_slt'][dm_select_idx]
select_dm_g['tang_lon'] = dm_g['tang_lon'][dm_select_idx]
select_dm_g['tang_lat'] = dm_g['tang_lat'][dm_select_idx]
select_dm_g['tang_alt'] = dm_g['tang_alt']

dm_select_idx=np.where((dm_r['time']>=select_dTN['time'][0]) & (dm_r['time'] < select_dTN['time'][-1]))
select_dm_r = {}
select_dm_r['orb_num'] = dm_r['orb_num'][dm_select_idx]
select_dm_r['time']    = dm_r['time'][dm_select_idx]
select_dm_r['uzon']    = dm_r['uzon'][dm_select_idx]
select_dm_r['umer']    = dm_r['umer'][dm_select_idx]
select_dm_r['tang_slt']      = dm_r['tang_slt'][dm_select_idx]
select_dm_r['tang_lon'] = dm_r['tang_lon'][dm_select_idx]
select_dm_r['tang_lat'] = dm_r['tang_lat'][dm_select_idx]
select_dm_r['tang_alt'] = dm_r['tang_alt']

orbit_start_time=pd.to_datetime(select_dTN['time'][0].values).strftime("%m-%d-%y %H:%M:%S")
orbit_end_time=pd.to_datetime(select_dTN['time'][-1].values).strftime("%m-%d-%y %H:%M:%S")

print('Selected time: %s to %s' % (orbit_start_time,orbit_end_time))

## Quick view of a single orbit L2.2 Vector Wind


*   Daytime observations of the red line (630.0 nm) emission span the altitude range of ~180 to 300 km while observations of the green line (557.7 nm) emission span the altitude range of 90 to ~190 km.  
*   Nighttime observations of the red line emission span the altitude range of ~210 to 300 km while those of the green line span 90 to ~109 km.
*   Daytime observations are provided at a 30-s cadence while nighttime observations are provided at a 60-s cadence.
*   How to deal with the overlapped altitudes? 








In [ ]:
X_g, Y_g = np.meshgrid(select_dm_g['time'],select_dm_g['tang_alt'])
X_r, Y_r = np.meshgrid(select_dm_r['time'],select_dm_r['tang_alt'])

show_time=pd.to_datetime(select_dm_g['time'].values)

# ## Wind ###

cmap='bwr'
vmax=200
vmin=-200
variable_name='MIGHTI L2.2 Meridional Wind'
cb_label='Meridional Wind (m/s)'
xlabel='UTC'
ylabel='Altitude (km)'

plt.close()
fig,ax=plt.subplots(figsize=(16,4))
fig.suptitle('%s %s' % (variable_name, show_time[0].strftime("%m-%d-%Y")),size=20,x=0.5)
gs = gridspec.GridSpec(1,2) 

ax0=plt.subplot(gs[0,0])
im = ax0.pcolor(X_g,Y_g,select_dm_g['umer'].transpose() ,cmap=cmap,vmin=vmin,vmax=vmax)
ax0.set_ylim([90,300])
ax0.set_title('Green')
ax0.set_xlabel(xlabel)
ax0.set_ylabel(ylabel)
color_bar=fig.colorbar(im)
color_bar.set_label(cb_label,size=16)


ax0=plt.subplot(gs[0,1])
im = ax0.pcolor(X_r,Y_r,select_dm_r['umer'].transpose() ,cmap=cmap,vmin=vmin,vmax=vmax)
ax0.set_ylim([90,300])
ax0.set_title('Red')
ax0.set_xlabel(xlabel)
ax0.set_ylabel(ylabel)
color_bar=fig.colorbar(im)
color_bar.set_label(cb_label,size=16)

fig.autofmt_xdate(rotation=45)


left  = 0.1  # the left side of the subplots of the figure
right = 0.9   # the right side of the subplots of the figure
bottom = 0.05   # the bottom of the subplots of the figure
top = 0.8     # the top of the subplots of the figure
wspace = 0.2#0.2   # the amount of width reserved for blank space between subplots
hspace = 0.3#0.2   # the amount of height reserved for white space between subplots

plt.subplots_adjust(left, bottom, right, top, wspace, hspace)



## Extension: Visualizing the neutral wind and temperature at mesosphere and the lower thermosphere at a time.

In [ ]:
## Temperature ###
target_1_array=select_dTN['TN']
X_1_array, _ = np.meshgrid(pd.to_datetime(select_dTN['time'].values),select_dTN['tang_alt'][10,:])

## Green Line Wind ###
target_2_array=select_dm_g['uzon']
X_2_array, Y_2_array = np.meshgrid(select_dm_g['time'],select_dm_g['tang_alt'])

# ## Temperature ###

cmap_1='plasma'
vmax_1=300
vmin_1=150
variable_name_1='ICON L2.3-%s Temperature' % (MTAB)
xlabel_1='UTC'
ylabel_1='Altitude (km)'
cb_label_1='Temperature (K)'
tang_alt_idx=5


# ## Wind ###

cmap_2='bwr'
vmax_2=200
vmin_2=-200
variable_name_2='ICON L2.2 Zonal Wind'
xlabel_2='UTC'
ylabel_2='Altitude (km)'
cb_label_2='Zonal Wind (m/s)'

######################## PLOT #############################
show_time=pd.to_datetime(select_dTN['time'].values)
fig,ax=plt.subplots(figsize=(8,8))#fig=pp.figure(figsize=(12,10)

fig.suptitle('UTC %s to %s' % (show_time[0].strftime("%H:%M:%S"),show_time[-1].strftime("%H:%M:%S %m-%d-%y")),size=20,x=0.43)
gs = gridspec.GridSpec(3, 1, height_ratios=[1,1,1]) 

ax0=plt.subplot(gs[0,0])
im = ax0.pcolor(X_1_array.transpose(),select_dTN['tang_alt'],target_1_array ,cmap=cmap_1,vmin=vmin_1,vmax=vmax_1)#,norm=colors.Normalize(vmin=0, vmax=150))
ax0.set_ylim([90,115])
ax0.set_title(variable_name_1)
ax0.set_xlabel(xlabel_1)
ax0.set_ylabel(ylabel_1)
color_bar=fig.colorbar(im)
color_bar.set_label(cb_label_1)

ax0=plt.subplot(gs[1,0])
im = ax0.pcolor(X_2_array,Y_2_array,target_2_array.transpose() ,cmap=cmap_2,vmin=vmin_2,vmax=vmax_2)#,norm=colors.Normalize(vmin=0, vmax=150))
ax0.set_ylim([90,115])
ax0.set_title(variable_name_2)
ax0.set_xlabel(xlabel_2)
ax0.set_ylabel(ylabel_2)
color_bar=fig.colorbar(im)
color_bar.set_label(cb_label_2)

ax0=plt.subplot(gs[2,0])
im=ax0.scatter(select_dTN['tang_lon'][tang_alt_idx,:],select_dTN['tang_lat'][tang_alt_idx,:],c=select_dTN['tang_slt'][tang_alt_idx,:],cmap='twilight_shifted')
ax0.set_title('Tangent point at ~%03i km' % select_dTN['tang_alt'][tang_alt_idx,1])
ax0.set_xlabel('Longitude')
ax0.set_ylabel('Latitude')

noon_idx=np.argmin(abs(select_dTN['tang_slt'][tang_alt_idx].values-12))
noon_lon=select_dTN['tang_lon'][tang_alt_idx,noon_idx]
noon_lat=select_dTN['tang_lat'][tang_alt_idx,noon_idx]
ax0.plot(noon_lon,noon_lat,marker='o',color='red',label='Noon time')
color_bar=fig.colorbar(im)
color_bar.set_label('Solar Local Time (hr)')

plt.tight_layout(rect=[0, 0.0, 1.05, 0.95])
fig.autofmt_xdate(rotation=45)

plt.legend()

left  = 0.1  # the left side of the subplots of the figure
right = 0.9   # the right side of the subplots of the figure
bottom = 0.05   # the bottom of the subplots of the figure
top = 0.9     # the top of the subplots of the figure
wspace = 0.1#0.2   # the amount of width reserved for blank space between subplots
hspace = 0.8#0.2   # the amount of height reserved for white space between subplots

plt.subplots_adjust(left, bottom, right, top, wspace, hspace)



# Example 2: Ionospheric measurements by EUV, FUV and IVM
- L2.5 FUV: Remotely sensed ***nighttime*** O+ profile, HmF2 and NmF2 on the FUV's limb.
- L2.6 EUV: Remotely sensed ***daytime*** O+ profile, HmF2 and NmF2 on the EUV's limb.
- L2.7 IVM: ***In-situ*** ion density, temperature, and composition at S/C altitude (~600 km)


In [ ]:

di_select_orb_num_idx = np.argwhere(di['orb_num'].values== target_orbit_num)
di_select_idx = di_select_orb_num_idx[:,0]

select_di = {}
select_di['orb_num'] = di['orb_num'][di_select_idx]
select_di['time']    = di['time'][di_select_idx]
select_di['Ni']      = di['Ni'][di_select_idx]
select_di['vmer']    = di['vmer'][di_select_idx]
select_di['slt']  = di['slt'][di_select_idx]
select_di['lon']  = di['lon'][di_select_idx]
select_di['lat']  = di['lat'][di_select_idx]
select_di['mlon'] = di['mlon'][di_select_idx]
select_di['mlat'] = di['mlat'][di_select_idx]


dEUV_select_idx=np.where((dEUV['time']>=select_di['time'][0]) & (dEUV['time'] < select_di['time'][-1]))
select_dEUV = {}
select_dEUV['orb_num'] = dEUV['orb_num'][dEUV_select_idx]
select_dEUV['time']    = dEUV['time'][dEUV_select_idx]
select_dEUV['O_plus']   = dEUV['O_plus'][dEUV_select_idx]
select_dEUV['HmF2']    = dEUV['HmF2'][dEUV_select_idx]
select_dEUV['NmF2']    = dEUV['NmF2'][dEUV_select_idx]

select_dEUV['tang_slt'] = dEUV['tang_slt'][dEUV_select_idx]
select_dEUV['tang_alt'] = dEUV['tang_alt'][dEUV_select_idx]

select_dEUV['tang_lon'] = dEUV['tang_lon'][dEUV_select_idx]
select_dEUV['tang_lat'] = dEUV['tang_lat'][dEUV_select_idx]
select_dEUV['tang_mlon'] = dEUV['tang_mlon'][dEUV_select_idx]
select_dEUV['tang_mlat'] = dEUV['tang_mlat'][dEUV_select_idx]

# dON2_select_idx=np.where((dON2['time']>=select_di['time'][0]) & (dON2['time'] < select_di['time'][-1]))
# select_dON2 = {}
# select_dON2['time']    = dON2['time'][dON2_select_idx]
# select_dON2['disk_ON2']     = dON2['disk_ON2'][dON2_select_idx]
# select_dON2['disk_slt']      = dON2['disk_slt'][dON2_select_idx]
# select_dON2['disk_lon'] = dON2['disk_lon'][dON2_select_idx]
# select_dON2['disk_lat'] = dON2['disk_lat'][dON2_select_idx]
# select_dON2['disk_mlon'] = dON2['disk_mlon'][dON2_select_idx]
# select_dON2['disk_mlat'] = dON2['disk_mlat'][dON2_select_idx]

dL25_select_idx=np.where((dL25['time']>=select_di['time'][0]) & (dL25['time'] < select_di['time'][-1]))
select_dL25 = {}
select_dL25['time']    = dL25['time'][dL25_select_idx]
select_dL25['HmF2']    = dL25['HmF2'][dL25_select_idx]
select_dL25['NmF2']    = dL25['NmF2'][dL25_select_idx]
select_dL25['HmF2_lon']    = dL25['HmF2_lon'][dL25_select_idx]
select_dL25['HmF2_lat']    = dL25['HmF2_lat'][dL25_select_idx]
select_dL25['HmF2_mlon']    = dL25['HmF2_mlon'][dL25_select_idx]
select_dL25['HmF2_mlat']    = dL25['HmF2_mlat'][dL25_select_idx]

select_dL25['tang_slt']    = dL25['tang_slt'][dL25_select_idx]

orbit_start_time=pd.to_datetime(select_di['time'][0].values).strftime("%m-%d-%y %H:%M:%S")
orbit_end_time=pd.to_datetime(select_di['time'][-1].values).strftime("%m-%d-%y %H:%M:%S")

print('Selected time: %s to %s' % (orbit_start_time,orbit_end_time))

## Visualizing HmF2 from EUV and FUV

In [ ]:
plt.close()

fig,ax=plt.subplots(figsize=(12,4))
fig.suptitle('ICON HmF2 %s' % ( show_time[0].strftime("%m-%d-%Y")),size=20,x=0.5)
gs = gridspec.GridSpec(1,2) 

ax0=plt.subplot(gs[0])
ax0.plot(select_dEUV['time'],select_dEUV['HmF2'],'ro',label='Daytime (EUV)')
ax0.plot(select_dL25['time'],np.nanmean(select_dL25['HmF2'],axis=1),'bo',label='Nighttime (FUV)')
ax0.set_xlabel('UTC')
ax0.set_ylabel('HmF2 (km)')
ax0.legend(fontsize=15)

ax0=plt.subplot(gs[1])
im=ax0.scatter(select_dEUV['time'].values,select_dEUV['tang_mlat'],c=select_dEUV['tang_slt'],vmax=24,vmin=0,cmap='twilight_shifted')
im=ax0.scatter(select_dL25['time'].values,select_dL25['HmF2_mlat'][:,3],c=select_dL25['tang_slt'][:,3],vmax=24,vmin=0,cmap='twilight_shifted')
ax0.set_ylabel('Mag. Latitude')
ax0.set_xlabel('UTC')
color_bar=fig.colorbar(im)
color_bar.set_label('Solar Local Time (hr)')

fig.autofmt_xdate(rotation=45)

plt.tight_layout(rect=[0, 0.0, 1.0, 0.95])


## Extension: Visualizing the in-situ IVM ion density and EUV O+ density profilt at tangent points

In [ ]:
show_time=pd.to_datetime(select_di['time'].values)
### IVM Ion Density ####

target_2_array=select_di['Ni']
X_2_array=select_di['time']

## EUV Oplus ###
target_1_array=select_dEUV['O_plus']
X_1_array, Y_1_array = np.meshgrid(select_dEUV['time'],select_dEUV['tang_alt'][0,:])


cmap_1='plasma'
vmax_1=8*10**5
vmin_1=1*10**5
variable_name_1='ICON L2.6 EUV Oplus %s' % show_time[0].strftime("%m-%d-%y")
xlabel_1='UTC'
ylabel_1='Altitude (km)'
cb_label_1='O+ Density (#/cm^3)'

######################## PLOT #############################

fig,ax=plt.subplots(figsize=(10,5))#fig=pp.figure(figsize=(12,10)


gs = gridspec.GridSpec(1, 1) 

ax0=plt.subplot(gs[0,0])
im = ax0.pcolor(X_1_array,Y_1_array,target_1_array.transpose() ,cmap=cmap_1,vmin=vmin_1,vmax=vmax_1)#,norm=colors.Normalize(vmin=0, vmax=150))
ax0.set_ylim([200,600])
ax0.set_xlim([show_time[0],show_time[-1]])
ax0.set_title(variable_name_1)
ax0.set_xlabel(xlabel_1)
ax0.set_ylabel(ylabel_1)
color_bar=fig.colorbar(im,format=OOMFormatter(5, mathText=True))
color_bar.set_label(cb_label_1)

ax2 = ax0.twinx()
ax2.plot(X_2_array,target_2_array/100000.,'g.',label='IVM Ion Density')
ax2.grid(None)
leg=plt.legend(loc='best')
for text in leg.get_texts():
    plt.setp(text, color = 'g',size=20)
ax2.tick_params(axis='y', colors="g")
ax0.set_xlim([show_time[0],show_time[-1]])

myFmt = mdates.DateFormatter('%H:%M')
plt.gca().xaxis.set_major_formatter(myFmt)
plt.tight_layout(rect=[0, 0.0, 1.05, 0.95])


# Example 3: IVM-MIGHTI  - Neutral wind driven electrodynamic interactions
- Once per orbit, ICON crosses the equator in the daytime.
- IVM measures the vertical drift at the equator
- MIGHTI measures the wind profile at the northern footpoint of the magnetic field line.


In [ ]:
'''The IVM-MIGHTI pair of equator crossing with minimum time difference during''' 
icon_near_equator = (abs(di['mlat']) < 1.) # ICON is within 1 degree of equator
daytime = abs(di['slt'] - 12) < 6 # ICON is in daytime hours
ii = np.where(icon_near_equator & daytime)
t0 = pd.to_datetime(di['time'][ii].values)
print('ICON near equator starting at %s' % t0[0])
dm=dm_g

dm_time=pd.to_datetime(dm['time'].values)

# Find MIGHTI index that matches this time
# Note: in practice, the true MIGHTI-IVM conjunction may happen a couple minutes before or after this time.
# Magnetic field line tracing is recommended to find true conjunctions.
# find_min is an array for the minimum time difference between each t0 and MIGHTI.
find_min=[np.nanmin(abs((dm_time-i).total_seconds())) if not np.all(np.isnan(dm['uzon'][np.argmin(abs((dm_time-i).total_seconds())),:])) else (np.nan) for i in t0]
# find_argmin is an array for the argument (index) of the minimum time difference between each t0 and MIGHTI.
find_argmin=[(np.argmin(abs((dm_time-i).total_seconds()))) if not np.all(np.isnan(dm['uzon'][np.argmin(abs((dm_time-i).total_seconds())),:])) else (np.nan) for i in t0]
# Transfer find_min (datetime.timedelta object) to a float array time_diff. 
time_diff=[(i) if not pd.isnull(i) else (np.nan) for i in find_min]

m_idx=find_argmin[np.nanargmin(time_diff)]
i_idx=np.nanargmin(time_diff)

In [ ]:

#### Plot the profiles ###
plt.close()
plt.figure(figsize=(6,8))

plt.plot(dm['uzon'][m_idx,:], dm['tang_alt'], 'C0-', label='Zonal Neutral Wind')
plt.plot(dm['umer'][m_idx,:], dm['tang_alt'], 'C1-', label='Meridional Neutral Wind')
plt.plot(di['vmer'][ii][i_idx], di['alt'][ii][i_idx], 'ks', label='Vertical Ion Velocity')
plt.ylabel('Altitude [km]')
plt.xlabel('Velocity [m/s]')
plt.title('Observations of neutral wind and ion velocity\non nearly identical magnetic field lines')
# plt.ylim([90,250])
plt.legend(loc='upper left')
plt.tight_layout(rect=[0, 0.0, 1.05, 0.95])